# Transforma em DataFrame e limpa

Leitura do DataFrame

In [ ]:
import pandas as pd

# Lê excel e transforma em DataFrame
vendas_df = pd.read_excel('../data/silver/2025_11_treated.xlsx')

Remove e renomeia colunas

In [ ]:
# Cria tupla de sufixos de colunas necessários
sufixos_necessários = tuple(['Vendas', 'Anúncios', 'Publicidade'])

# Define lista de colunas do dataframe e remove 'Reclamação_encerrada_Reclamações' da lista pois é uma exceção a regra definida e será necessária
colunas_df = list(vendas_df.columns)
colunas_df.remove('Reclamação_encerrada_Reclamações')

# Remove colunas desnecessárias pelo sufixo
for col in colunas_df:
    if not col.endswith(sufixos_necessários):
        vendas_df = vendas_df.drop(col, axis=1)

# Dropa mais colunas desnecessárias
colunas_drop = ['Descrição_do_status_Vendas', 'Pacote_de_diversos_produtos_Vendas', 'Pertence_a_um_kit_Vendas', 'SKU_Anúncios', '#_de_anúncio_Anúncios', 'Canal_de_venda_Anúncios', 'Receita_por_acréscimo_no_preço_(pago_pelo_comprador)_Vendas', 'Taxa_de_parcelamento_equivalente_ao_acréscimo_Vendas']
vendas_df = vendas_df.drop(colunas_drop, axis=1)

In [ ]:
# Renomeia as colunas
novos_nomes = ['id_venda', 'data_venda', 'status_venda', 'unidades_vendidas', 'receita_por_produto', 'tarifas_impostos_venda', 'receita_envio', 'tarifa_envio', 'custo_envio', 'diff_custo_envio', 'cancelamentos_reembolsos', 'total_vendas', 'mes_faturamento_tarifas', 'venda_publicidade', 'titulo_anuncio', 'variacao_anuncio', 'preco_unitario_anuncio', 'tipo_anuncio', 'reclamacao_encerrada']
vendas_df.columns = novos_nomes

Filtra apenas vendas concluídas

In [ ]:
import numpy as np

# Cria lista auxiliar que crirá coluna count_produto
count_produto = []

# Para cada linha do df, checa se existe uma condição que deve anular a contagem do produto
# Como todas as linhas são necessárias para análise, cria uma coluna 'count_produto' e marca com False nesses casos
for row in vendas_df.itertuples():
    if (row.total_vendas <= 0) or ('cancel' in row.status_venda.lower()) or ((row.reclamacao_encerrada == 1) & (np.isnan(row.total_vendas))):
        count_produto.append(False)
    else:
        count_produto.append(True)

# Cria coluna count_produto
vendas_df['count_produto'] = count_produto

Corrige tipos e valores

In [ ]:
# Converte o id de int para string
vendas_df['id_venda'] = vendas_df['id_venda'].astype(str)

In [ ]:
# Remove a hora da coluna de data
vendas_df['data_venda'] = vendas_df['data_venda'].str[:-9]

In [ ]:
# Substitui ponto por vírgula nas colunas numéricas
for column in vendas_df.columns:

    # A coluna unidades_vendidas deve ser int
    if column in ['unidades_vendidas', 'reclamacao_encerrada']:
        vendas_df[column] = vendas_df[column].fillna(0).astype(int)

    # As numéricas restantes deverão se tornar strings com casa decimal separada por vírgula
    elif vendas_df[column].dtype == 'float64':
        vendas_df[column] = vendas_df[column].fillna(0).astype(str).str.replace('.', ',', regex=False)

In [ ]:
# Remove os espaços vazios da coluna de variação do anuncio e as chaves 'cor' e 'tamanho'
vendas_df['variacao_anuncio'] = vendas_df['variacao_anuncio'] \
                                    .str.replace(' ', '', regex=False) \
                                    .str.replace('Cor:', '', regex=False) \
                                    .str.replace('Tamanho:', '', regex=False)

In [ ]:
# Cria as colunas cor kimono e tamanho kimono a partir da coluna variação anúncio
vendas_df[['cor_kimono', 'tamanho_kimono']] = vendas_df['variacao_anuncio'].str.split('|', expand=True)

In [ ]:
# Salva DataFrame
vendas_df.to_csv('/../data/gold/2025_11_final.csv', index=False)